In [1]:
a = {
    "buildship": "https://github.com/eclipse/steady.git",
    "eclips-collections": "https://github.com/eclipse/eclipse-collections.git",
    "hawkbit": "https://github.com/eclipse/hawkbit.git",

    "kura": "https://github.com/eclipse/kura.git",
    # "milo" "https://github.com/eclipse/milo.git",
    "openvsx": "https://github.com/eclipse/openvsx.git",
    "steady": "https://github.com/eclipse/steady.git",
    "xtext": "https://github.com/eclipse/xtext.git"
    

}

In [3]:
import is_pos_to_neg
for p,u in a.items():

    is_pos_to_neg.is_modified(p)
 

void
int


TypeError: argument of type 'float' is not iterable

In [2]:
import pandas as pd
import json
json_path = 'projects_list.json'
with open(json_path, 'r') as f:
    projects = json.load(f)
for project,url in a.items():
    df2 = pd.read_csv(f'../data/{project}_is_labeled.csv')

    df1 = pd.read_csv(f'../data/{project}_is_labeled_temp.csv')
    df1 = df1[df1['label'] == 'negative']
    df2 = df2[df2['label'] == 'positive']
    combined = pd.concat([df1, df2])
    combined.to_csv(f'../data/{project}_is_combined.csv', index=False)
    
     
    

FileNotFoundError: [Errno 2] No such file or directory: '../data/buildship_is_labeled_temp.csv'

combine data

In [19]:
project = 'buildship'
df1 = pd.read_csv(f'../data/{project}_is_combined.csv')
df2 = pd.read_csv(f'../data/{project}_set_combined.csv')
df3 = pd.read_csv(f'../data/{project}_get_combined.csv')
df1['catagory'] = 'is'
df2['catagory'] = 'set'
df3['catagory'] = 'get'
for index, row in df1.iterrows():
    if row['label'] == 'negative':
        df1.at[index, 'catagory'] = 'not_smeel'
        
for index, row in df2.iterrows():
    if row['label'] == 'negative':
        df2.at[index, 'catagory'] = 'not_smeel'
        
for index, row in df3.iterrows():
    if row['label'] == 'negative':
        df3.at[index, 'catagory'] = 'not_smeel'

combined = pd.concat([df1, df2, df3])

In [20]:
combined=combined.drop(columns=['label'],axis=1)

In [22]:
# combined.head()
from sklearn.utils import shuffle
combined = shuffle(combined)

In [23]:
combined.to_csv(f'../data/{project}_combined.csv', index=False)

In [23]:
import ktrain
from ktrain import text

MODEL_NAME = 'microsoft/codebert-base'
t = text.Transformer(MODEL_NAME, maxlen=512, class_names=['not_smeel', 'is','set'])

In [5]:
import pandas as pd
import numpy as np
project = 'buildship'
df = pd.read_csv(f'./{project}_combined.csv')
df = df.drop(columns=["comments"],axis=1)
df = df.dropna(subset=['content'])
# access_modifier,return_type,method_name,parameters,returns,comments,content,label

df['combined'] = df['access_modifier']+' '+df['return_type']+' '+df['method_name']+'('+df['parameters']+')'+df['content']
# y = df['label'].map({'positive': 1, 'negative': 0}).values

In [6]:
X = df['combined'].tolist()
y = df['catagory'].to_list()

In [22]:
df['catagory'].value_counts()

catagory
set          1700
not_smeel     616
is             23
Name: count, dtype: int64

In [7]:
# Manually split your dataset into training and validation sets if not already done
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [8]:
print(len(X_train), len(X_val))

2105 234


In [9]:
# Note: Adjust maxlen according to your data's characteristics
trn = text.Transformer(MODEL_NAME, maxlen=512, class_names=['not_smeel', 'is','get','set'])
train_data = trn.preprocess_train(X_train, y_train)
val_data = trn.preprocess_test(X_val, y_val)

preprocessing train...
language: en
train sequence lengths:
	mean : 27
	95percentile : 94
	99percentile : 193


/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/utils.py:744: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 76
	99percentile : 136


In [10]:
# Get classifier model
model = trn.get_classifier()

# Create a Learner object
learner = ktrain.get_learner(model, train_data=train_data, val_data=val_data, batch_size=6)

In [16]:
# Train the model
learner.fit_onecycle(5e-5, 1)  # Adjust epochs and learning rate as needed



begin training using onecycle policy with max lr of 5e-05...
351/351 [==============================] - 4418s 13s/step - loss: 0.1404 - accuracy: 0.9544 - val_loss: 0.0996 - val_accuracy: 0.9744


In [11]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.save('./code_bert_model')

In [12]:
learner.validate(class_names=['not_smeel', 'is','get','set'])

      2/Unknown - 26s 12s/step

KeyboardInterrupt: 

In [20]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.save('./code_bert_model')

In [12]:
predictor.save('./code_bert_model')

In [24]:
reloaded_predictor = ktrain.load_predictor('./code_bert_model')

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/text/preprocessor.py:392: UserWarning: The method preprocess_train was never called. You can disable this warning by setting preprocess_train_called=True.
  warnings.warn(
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/text/preprocessor.py:1093: UserWarning: Could not load a Tensorflow version of model. (If this worked before, it might be an out-of-memory issue.) Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn(


ValueError: cannot reshape array of size 2304 into shape (768,4)

In [17]:
predictor.predict('public void setupRegistry() { stateBeforeInjectorCreation = GlobalRegistries.makeCopyOfGlobalState(); if (injector == null) { getInjector(); } stateAfterInjectorCreation.restoreGlobalState(); }')

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/text/preprocessor.py:392: UserWarning: The method preprocess_train was never called. You can disable this warning by setting preprocess_train_called=True.
  warnings.warn(


'is'

In [19]:
predictor.explain("public long getCacheMiss() { return this.cacheMissCount; }")

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/text/preprocessor.py:392: UserWarning: The method preprocess_train was never called. You can disable this warning by setting preprocess_train_called=True.
  warnings.warn(
